In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [4]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file=None, node_file=None):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
#        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
#         self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
            df = pd.read_excel(fileName, sheet_name = None)
            self.N_number = df["coordinates"].index[-1]+1
            self.N = set([i for i in range(1, self.N_number+1)])
            print(self.N)
            for i in df["fire_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                time = i['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
            for i in self.N:
                self.A_p.append((i,i))
            for i in df["firefighter_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                firefighterIndex = i['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = i['travel time']
                self.tau[u,v,firefighterIndex] = time
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v)) 
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(df["coordinates"].iloc[i]['x']), int(df["coordinates"].iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(df["coordinates"].iloc[i]['quantity'])
                    self.b[i+1] = int(df["coordinates"].iloc[i]['value'])
                    self.H[i+1] = int(df["coordinates"].iloc[i]['burning time'])
            print(len(self.H))
            self.N_D = set(df['ff_source']['N_D'].tolist())
            self.N_F = set(df['fire_source']['N_F'].tolist())
        
        #         if "fire_route" in fileName:
#             fire_df = pd.read_excel(fileName)
#             df_num = len(fire_df.index)
#             for i in range(df_num):
#                 u = int(fire_df.iloc[i]['i'])
#                 v = int(fire_df.iloc[i]['j'])
#                 time = fire_df.iloc[i]['travel time']
#                 self.lamb[u,v] = time
#                 self.A_f.append((u,v))
#         elif "firefighter_route" in fileName:
#             firefighter_df = pd.read_excel(fileName)
#             df_num = len(firefighter_df.index)
#             for i in self.N:
#                 self.A_p.append((i,i))
#             for i in range(df_num):
#                 u = int(firefighter_df.iloc[i]['i'])
#                 v = int(firefighter_df.iloc[i]['j'])
#                 firefighterIndex = firefighter_df.iloc[i]['k']
#                 if firefighterIndex not in self.K:
#                     self.K.add(int(firefighterIndex))
#                 time = firefighter_df.iloc[i]['travel time']
#                 self.tau[u,v,firefighterIndex] = time
#                 #避免在多消防員時重複紀錄arc set
#                 if (u,v) not in self.A_p:
#                     self.A_p.append((u,v))  
#         elif "nodeInformation" in fileName:
#             nodeInfo1 = pd.read_excel(fileName, 'coordinates')
#             df_num = len(nodeInfo1.index)
#             self.N_number = df_num
#             self.N = set([i for i in range(1, self.N_number+1)])
            
#             for i in range(self.N_number):
#                 self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
#             if self.mode == 'random':
#                 for i in range(self.N_number):
#                     self.Q[i+1] = random.randint(2,5)
#                     self.b[i+1] = random.randint(20,30)
#                     self.H[i+1] = random.randint(2,5)
#             else:
#                 for i in range(self.N_number):
#                     self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
#                     self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
#                     self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
#             nodeInfo2 = pd.read_excel(fileName, 'source')
#             self.N_D = set(nodeInfo2['N_D'].tolist())
#             self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

        print("w:")
        for (i, k, t) in self.w:
            if self.w[i, k, t].X > self.epsilon:
                print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

        print("u:")
        for (i, t) in self.u:
            if self.u[i,t].X > self.epsilon:
                print("u[%d,%d]" % (i,t), self.u[i,t].X)

        print("u_bar:")
        for (i, k, t) in self.u_bar:
            if self.u_bar[i, k, t].X > self.epsilon:
                print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

        print("v:")
        for (i, t) in self.v:
            if self.v[i, t].X > self.epsilon:
                print("v[%d,%d]" % (i,t), self.v[i, t].X)

        print("v_bar:")
        for (i, t) in self.v_bar:
            if self.v_bar[i, t].X > self.epsilon:
                print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self, file):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open(file[:-5]+"_data.json", "w") as file:
            file.write(json.dumps(data))

In [5]:
import os
files = os.listdir("./")

file_list = [f for f in files if os.path.isfile(os.path.join("./", f))and f[-5:] == ".xlsx"]

for file in file_list:
    print(file)

    ff = firefighterProblem(fire_file=file)
    model = ff.initialize()

    #run model and write lp file
    model.optimize()
    #model.write('test.lp')
    #print("optimal value : ", model.ObjVal)

    #ff.showTextSol()
    ff.writeJson(file)

G30_first_model.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
30
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 53702 rows, 47112 columns and 185720 nonzeros
Model fingerprint: 0xd5e2db00
Variable types: 0 continuous, 47112 integer (47112 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 39171 rows and 26348 columns
Presolve time: 2.67s
Presolved: 14531 rows, 20764 columns, 63230 nonzeros
Variable types: 0 continuous, 20764 integer (20726 binary)
Found heuristic solution: objective 54.0020000

Root relaxation: objective 1.001422e+01, 1021 iterations, 0.10 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

     0     0  189.31099    0  804  205.02850  189.31099  7.67%     -   22s
     0     0  189.31152    0  800  205.02850  189.31152  7.67%     -   22s
     0     0  189.98998    0  788  205.02850  189.98998  7.33%     -   23s
     0     0  189.99880    0  775  205.02850  189.99880  7.33%     -   23s
     0     0  189.99880    0  779  205.02850  189.99880  7.33%     -   23s
     0     0  190.33611    0  809  205.02850  190.33611  7.17%     -   24s
     0     0  190.39989    0  823  205.02850  190.39989  7.13%     -   24s
     0     0  190.39990    0  786  205.02850  190.39990  7.13%     -   25s
     0     0  190.42746    0  934  205.02850  190.42746  7.12%     -   25s
     0     0  190.45369    0  940  205.02850  190.45369  7.11%     -   26s
     0     0  190.45717    0  947  205.02850  190.45717  7.11%     -   26s
     0     0  190.45867    0  933  205.02850  190.45867  7.11%     -   26s
     0     0  190.56111    0  945  205.02850  190.56111  7.06%     -   26s
     0     0  190.56937  


Root relaxation: objective 5.556263e+01, 10399 iterations, 4.92 seconds (2.26 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   55.56263    0  647  240.00150   55.56263  76.8%     -    8s
H    0     0                     205.0269000   55.56263  72.9%     -   10s
     0     0   60.94605    0  679  205.02690   60.94605  70.3%     -   13s
     0     0   68.98771    0  565  205.02690   68.98771  66.4%     -   17s
     0     0   71.10497    0  617  205.02690   71.10497  65.3%     -   21s
     0     0   71.10500    0  629  205.02690   71.10500  65.3%     -   21s
H    0     0                     195.0295000   71.10500  63.5%     -   24s
H    0     0                     185.0283000   71.10500  61.6%     -   24s
H    0     0                     160.0399000   71.10500  55.6%     -   24s
H    0     0                     160.0381000   71.10500  55.6%     -   24s
     0 

  1580   469     cutoff   49       150.02730  120.02551  20.0%   400  376s
H 1615   410                     145.0392000  120.02555  17.2%   397  376s
H 1632   410                     145.0388000  120.67674  16.8%   399  376s
  1633   409     cutoff   21       145.03880  121.25129  16.4%   400  387s
  1681   406  142.04370   29  100  145.03880  122.19789  15.7%   400  397s
  1777   405  142.81763   32  235  145.03880  124.32447  14.3%   397  407s
  1857   431     cutoff   35       145.03880  124.58348  14.1%   398  417s
  1998   446  137.80242   34  397  145.03880  127.20396  12.3%   388  427s
  2141   438  133.96058   35  252  145.03880  128.49376  11.4%   380  438s
  2258   416 infeasible   44       145.03880  130.02435  10.4%   381  450s
  2433   330 infeasible   42       145.03880  131.75948  9.16%   372  473s
  2648   126     cutoff   31       145.03880  135.02535  6.90%   362  490s

Cutting planes:
  Gomory: 3
  Cover: 12
  Implied bound: 3
  Clique: 21
  MIR: 7
  StrongCG: 1
  In

   201   146  104.62031   38  703  155.02690   99.13267  36.1%   393   92s
   214   149  104.00610   38  632  155.02690   99.13267  36.1%   389   95s
   242   160  112.75623   40  759  155.02690   99.13267  36.1%   398  100s
   257   163  147.43435   42  537  155.02690   99.13267  36.1%   428  106s
   280   163     cutoff   43       155.02690   99.52074  35.8%   454  113s
   290   157  151.90835    5  218  155.02690  100.05785  35.5%   466  116s
   316   158     cutoff   23       155.02690  100.25257  35.3%   471  120s
   333   159  109.45952   29  698  155.02690  100.38725  35.2%   511  127s
   361   186  127.72890   34  459  155.02690  100.38725  35.2%   515  130s
   376   214  134.38339   36  426  155.02690  100.41289  35.2%   506  136s
   446   240  150.02868   30  168  155.02690  102.31829  34.0%   473  142s
   491   252     cutoff   82       155.02690  102.74322  33.7%   451  145s
   553   281  107.38618   41  775  155.02690  102.75129  33.7%   427  152s
   577   311  127.55584  

     0     0  165.17515    0  397  230.03440  165.17515  28.2%     -   18s
     0     0  165.51251    0  425  230.03440  165.51251  28.0%     -   20s
     0     0  165.56270    0  423  230.03440  165.56270  28.0%     -   20s
     0     0  166.47613    0  518  230.03440  166.47613  27.6%     -   21s
H    0     0                     230.0341000  166.47613  27.6%     -   21s
     0     0  166.55675    0  520  230.03410  166.55675  27.6%     -   21s
     0     0  167.19247    0  552  230.03410  167.19247  27.3%     -   22s
     0     0  167.39558    0  535  230.03410  167.39558  27.2%     -   23s
     0     0  167.39559    0  537  230.03410  167.39559  27.2%     -   23s
     0     0  167.96020    0  564  230.03410  167.96020  27.0%     -   23s
     0     0  167.99113    0  536  230.03410  167.99113  27.0%     -   24s
     0     0  168.01007    0  548  230.03410  168.01007  27.0%     -   24s
     0     0  168.01375    0  530  230.03410  168.01375  27.0%     -   24s
     0     0  168.69914  

H    0     0                     175.0361000  129.26934  26.1%     -   20s
H    0     0                     155.0321000  129.26934  16.6%     -   20s
     0     0  129.26934    0  493  155.03210  129.26934  16.6%     -   20s
     0     0  129.92126    0  543  155.03210  129.92126  16.2%     -   24s
     0     0  130.35871    0  623  155.03210  130.35871  15.9%     -   27s
     0     0  130.39235    0  614  155.03210  130.39235  15.9%     -   28s
     0     0  130.39235    0  615  155.03210  130.39235  15.9%     -   28s
     0     0  130.39979    0  622  155.03210  130.39979  15.9%     -   28s
     0     0  130.39983    0  628  155.03210  130.39983  15.9%     -   28s
     0     0  130.67741    0  749  155.03210  130.67741  15.7%     -   29s
     0     0  130.67841    0  750  155.03210  130.67841  15.7%     -   30s
     0     0  131.12753    0  696  155.03210  131.12753  15.4%     -   30s
     0     0  131.18030    0  747  155.03210  131.18030  15.4%     -   31s
     0     0  131.18881  

     0     0  148.93263    0 1042  155.03210  148.93263  3.93%     -  134s
     0     0  148.93263    0  218  155.03210  148.93263  3.93%     -  138s
     0     0  148.93263    0  661  155.03210  148.93263  3.93%     -  141s
     0     0  148.93263    0  721  155.03210  148.93263  3.93%     -  142s
     0     0  148.93263    0  796  155.03210  148.93263  3.93%     -  143s
     0     0  148.93599    0  896  155.03210  148.93599  3.93%     -  143s
     0     0  149.01496    0  866  155.03210  149.01496  3.88%     -  143s
     0     0  149.03984    0  900  155.03210  149.03984  3.87%     -  143s
     0     0  149.04629    0  911  155.03210  149.04629  3.86%     -  144s
     0     0  149.93810    0 1007  155.03210  149.93810  3.29%     -  146s
     0     0  150.00295    0 1006  155.03210  150.00295  3.24%     -  146s
     0     0  150.00569    0 1003  155.03210  150.00569  3.24%     -  146s
     0     0  150.44045    0  943  155.03210  150.44045  2.96%     -  148s
     0     0  150.44045  

H    0     0                     155.0292000  107.87821  30.4%     -   45s
     0     2  107.98938    0  849  155.02920  107.98938  30.3%     -   46s
     1     2  108.09022    1  783  155.02920  107.99080  30.3%   706   50s
     3     4  108.13034    2  750  155.02920  108.09022  30.3%  2033   55s
     5     6  108.13650    3  758  155.02920  108.13650  30.2%  1659   60s
    13    10  108.15323    5  760  155.02920  108.13915  30.2%  1678   65s
    21    17  108.28350    7  787  155.02920  108.13915  30.2%  1194   71s
    33    27  111.15814   10  604  155.02920  108.13915  30.2%  1018   75s
    60    52  110.43950   15  707  155.02920  108.13915  30.2%   672   81s
    89    57  111.41955   18  700  155.02920  108.13915  30.2%   548   87s
    95    56  111.44038   19  727  155.02920  108.13915  30.2%   558   90s
   108    60  111.98083   22  704  155.02920  108.13915  30.2%   563   97s
   132    66  112.42080   28  699  155.02920  108.13915  30.2%   574  102s
   146    79  113.87558  